In [1]:
import os
import glob
import re
import numpy as np
import numpy.fft as fft
import mne
import matplotlib.pyplot as plt
from scipy import stats, signal
from numpy import save, load
import pandas as pd
import sklearn
from sklearn.svm import SVC

### read dataset X and Y

In [2]:
subject_id=1
base_path = "features_notwelch/"
# edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb01/*.npy".format(subject_id))))
# files=len(edf_file_names)
X=load('features_notwelch/data_chb{:02d}/features_{}_00.npy'.format(subject_id,subject_id))
y=load('features_notwelch/data_chb{:02d}/targets_{}_00.npy'.format(subject_id,subject_id))
for subject_id in (1,3,6,7,8):
    edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb{:02d}/*.npy".format(subject_id))))
    files=len(edf_file_names)
    start=0
    if subject_id==1:
        start=1
    for fileno in range(start, files//2):
        X=np.concatenate((X, load('features_notwelch/data_chb{:02d}/features_{}_{:02d}.npy'.format(subject_id, subject_id, fileno))))
        y=np.concatenate((y, load('features_notwelch/data_chb{:02d}/targets_{}_{:02d}.npy'.format(subject_id,subject_id, fileno))))
    print(files)
    print(X.shape)


84
(14577, 208)
76
(28145, 208)
36
(52073, 208)
38
(76155, 208)
40
(83290, 208)


In [3]:
X_shape, y_shape = X.shape, y.shape
print(X_shape, y_shape)

(83290, 208) (83290,)


In [4]:
# def generate_columns():
#     channel_order= {'FP1-F7':0, 'F7-T7':1, 'T7-P7':2, 'P7-O1':3, 'FP1-F3':4, 'F3-C3':5, 'C3-P3':6, 'P3-O1':7, 'FP2-F4':8, 'F4-C4':9, 'C4-P4':10, 'P4-O2':11, 'FP2-F8':12, 'F8-T8':13, 'T8-P8-0':14, 'P8-O2':15, 'FZ-CZ':16, 'CZ-PZ':17, 'P7-T7':18, 'T7-FT9':19, 'FT9-FT10':20, 'FT10-T8':21 ,'T8-P8-1':22 }
#     col_names=[]
#     for ch in channel_order:
#         # print(ch)
#         for i in range(0,9):
#             col_names.append(str(ch)+'-'+str(i))
#     col_names.append('patient')
#     return col_names

def generate_column_names():
    channel_order= {'FP1-F7':0, 'F7-T7':1, 'T7-P7':2, 'P7-O1':3, 'FP1-F3':4, 'F3-C3':5, 'C3-P3':6, 'P3-O1':7, 'FP2-F4':8, 'F4-C4':9, 'C4-P4':10, 'P4-O2':11, 'FP2-F8':12, 'F8-T8':13, 'T8-P8-0':14, 'P8-O2':15, 'FZ-CZ':16, 'CZ-PZ':17, 'P7-T7':18, 'T7-FT9':19, 'FT9-FT10':20, 'FT10-T8':21 ,'T8-P8-1':22 }
    col_names=[]
    for ch in channel_order:
        # print(ch)
        for i in range(0,9):
            col_names.append(str(ch)+'-'+str(i))
    # col_names.append('rms')
    # col_names.append('sum')
    col_names.append('patient')
    # print(col_names.shape)
    return col_names


In [5]:
df=pd.DataFrame(data=X, columns=generate_column_names())
# df['patient']=1
# df.at[14577:27187,'patient',]=2
# df.at[27187:40755, 'patient']=3
# df.at[40755:97783, 'patient']=4
# df.at[97783:, 'patient']=5

# df=df[np.append(df.columns[:50], ['rms', 'sum', 'patient'])]
df['target']=y
print(df.shape)
df=df.dropna()
print(df.shape)
df.reset_index(drop=True, inplace=True)

(83290, 209)
(83290, 209)


In [6]:
df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-1,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,target
0,0.009253,0.205576,0.015386,0.013003,0.353573,0.068100,0.261663,0.048393,13087.628906,0.009094,...,0.160299,0.006780,0.013950,0.344800,0.081632,0.294163,0.059692,2853.293701,1.0,0
1,0.010184,0.198871,0.017687,0.014204,0.371332,0.078176,0.244087,0.050380,13272.797852,0.009179,...,0.166040,0.009536,0.017124,0.360151,0.080480,0.282719,0.057230,2595.387695,1.0,0
2,0.013337,0.209899,0.024952,0.019939,0.382135,0.072422,0.219988,0.053930,9027.048828,0.010076,...,0.128410,0.010422,0.017145,0.405914,0.107520,0.246479,0.072111,2669.577881,1.0,0
3,0.014367,0.214465,0.024949,0.024575,0.367699,0.068725,0.233418,0.046182,6688.242676,0.017899,...,0.138748,0.010597,0.014635,0.405534,0.109894,0.233375,0.072846,2570.697998,1.0,0
4,0.024251,0.179508,0.064263,0.053937,0.377249,0.069827,0.200093,0.030606,4267.430176,0.043423,...,0.160317,0.014761,0.022323,0.309459,0.085600,0.301347,0.069259,1922.287476,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83285,0.008805,0.162759,0.021875,0.016082,0.349363,0.080733,0.279748,0.066814,14735.153320,0.004093,...,0.209121,0.008912,0.010706,0.424271,0.065908,0.208240,0.045127,26926.115234,8.0,0
83286,0.017645,0.218688,0.020033,0.019581,0.390985,0.076732,0.217819,0.045547,6259.095703,0.003820,...,0.202268,0.006331,0.007070,0.421968,0.061125,0.217734,0.052048,43083.664062,8.0,0
83287,0.016607,0.104798,0.030455,0.021936,0.281037,0.185331,0.306219,0.074334,7056.773926,0.003916,...,0.209323,0.004378,0.005458,0.428714,0.052781,0.217020,0.056849,34002.648438,8.0,0
83288,0.014434,0.072497,0.041453,0.022231,0.244569,0.183943,0.329688,0.089199,7828.537109,0.005187,...,0.238687,0.006064,0.007115,0.425626,0.044030,0.216710,0.049417,21102.750000,8.0,0


In [7]:
df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-1,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,target
0,0.009253,0.205576,0.015386,0.013003,0.353573,0.068100,0.261663,0.048393,13087.628906,0.009094,...,0.160299,0.006780,0.013950,0.344800,0.081632,0.294163,0.059692,2853.293701,1.0,0
1,0.010184,0.198871,0.017687,0.014204,0.371332,0.078176,0.244087,0.050380,13272.797852,0.009179,...,0.166040,0.009536,0.017124,0.360151,0.080480,0.282719,0.057230,2595.387695,1.0,0
2,0.013337,0.209899,0.024952,0.019939,0.382135,0.072422,0.219988,0.053930,9027.048828,0.010076,...,0.128410,0.010422,0.017145,0.405914,0.107520,0.246479,0.072111,2669.577881,1.0,0
3,0.014367,0.214465,0.024949,0.024575,0.367699,0.068725,0.233418,0.046182,6688.242676,0.017899,...,0.138748,0.010597,0.014635,0.405534,0.109894,0.233375,0.072846,2570.697998,1.0,0
4,0.024251,0.179508,0.064263,0.053937,0.377249,0.069827,0.200093,0.030606,4267.430176,0.043423,...,0.160317,0.014761,0.022323,0.309459,0.085600,0.301347,0.069259,1922.287476,1.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83285,0.008805,0.162759,0.021875,0.016082,0.349363,0.080733,0.279748,0.066814,14735.153320,0.004093,...,0.209121,0.008912,0.010706,0.424271,0.065908,0.208240,0.045127,26926.115234,8.0,0
83286,0.017645,0.218688,0.020033,0.019581,0.390985,0.076732,0.217819,0.045547,6259.095703,0.003820,...,0.202268,0.006331,0.007070,0.421968,0.061125,0.217734,0.052048,43083.664062,8.0,0
83287,0.016607,0.104798,0.030455,0.021936,0.281037,0.185331,0.306219,0.074334,7056.773926,0.003916,...,0.209323,0.004378,0.005458,0.428714,0.052781,0.217020,0.056849,34002.648438,8.0,0
83288,0.014434,0.072497,0.041453,0.022231,0.244569,0.183943,0.329688,0.089199,7828.537109,0.005187,...,0.238687,0.006064,0.007115,0.425626,0.044030,0.216710,0.049417,21102.750000,8.0,0


In [8]:
df['seizure_number']=0
df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,target,seizure_number
0,0.009253,0.205576,0.015386,0.013003,0.353573,0.068100,0.261663,0.048393,13087.628906,0.009094,...,0.006780,0.013950,0.344800,0.081632,0.294163,0.059692,2853.293701,1.0,0,0
1,0.010184,0.198871,0.017687,0.014204,0.371332,0.078176,0.244087,0.050380,13272.797852,0.009179,...,0.009536,0.017124,0.360151,0.080480,0.282719,0.057230,2595.387695,1.0,0,0
2,0.013337,0.209899,0.024952,0.019939,0.382135,0.072422,0.219988,0.053930,9027.048828,0.010076,...,0.010422,0.017145,0.405914,0.107520,0.246479,0.072111,2669.577881,1.0,0,0
3,0.014367,0.214465,0.024949,0.024575,0.367699,0.068725,0.233418,0.046182,6688.242676,0.017899,...,0.010597,0.014635,0.405534,0.109894,0.233375,0.072846,2570.697998,1.0,0,0
4,0.024251,0.179508,0.064263,0.053937,0.377249,0.069827,0.200093,0.030606,4267.430176,0.043423,...,0.014761,0.022323,0.309459,0.085600,0.301347,0.069259,1922.287476,1.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83285,0.008805,0.162759,0.021875,0.016082,0.349363,0.080733,0.279748,0.066814,14735.153320,0.004093,...,0.008912,0.010706,0.424271,0.065908,0.208240,0.045127,26926.115234,8.0,0,0
83286,0.017645,0.218688,0.020033,0.019581,0.390985,0.076732,0.217819,0.045547,6259.095703,0.003820,...,0.006331,0.007070,0.421968,0.061125,0.217734,0.052048,43083.664062,8.0,0,0
83287,0.016607,0.104798,0.030455,0.021936,0.281037,0.185331,0.306219,0.074334,7056.773926,0.003916,...,0.004378,0.005458,0.428714,0.052781,0.217020,0.056849,34002.648438,8.0,0,0
83288,0.014434,0.072497,0.041453,0.022231,0.244569,0.183943,0.329688,0.089199,7828.537109,0.005187,...,0.006064,0.007115,0.425626,0.044030,0.216710,0.049417,21102.750000,8.0,0,0


In [9]:
seizure_number=1
for i in range(1, len(df)):
    # if df.iloc[i]['target']==2 and df.iloc[i-1]['target']==2:
    #     df.iloc[i]['seizure_number']=df.iloc[i-1]['seizure_number']
    if df.iloc[i]['target']==2:
        df.at[i,'seizure_number']=seizure_number
    if (df.iloc[i]['target']!=2 and df.iloc[i-1]['target']==2 ):
        seizure_number+=1
# df.to_csv(r'df.csv')


In [10]:
test_df=pd.DataFrame()
for patient in pd.unique(df['patient']):
    # patdf=df[df[207]==patient]
    last_seizure=max(df[df['patient']==patient]['seizure_number'])
    test_df = test_df.append(df[(df['patient']==patient) & (df['seizure_number']==last_seizure)], ignore_index=False)
test_df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,target,seizure_number
9049,0.487906,0.294157,0.062645,0.243455,0.061901,0.003636,0.013767,0.003427,733.779602,0.385181,...,0.066628,0.112517,0.115718,0.304856,0.541955,0.017563,1923.753174,1.0,2,7
9050,0.492928,0.287013,0.072026,0.238879,0.060910,0.002932,0.012587,0.004014,711.672302,0.415166,...,0.068790,0.102843,0.090541,0.174525,0.623085,0.014286,2015.595825,1.0,2,7
9051,0.406936,0.237590,0.078928,0.220020,0.104659,0.004607,0.020451,0.006975,398.291138,0.365368,...,0.043970,0.070681,0.184607,0.131209,0.573714,0.013855,1690.714233,1.0,2,7
9052,0.377607,0.232304,0.089268,0.232302,0.118813,0.004778,0.024522,0.006443,364.721436,0.288228,...,0.031371,0.090012,0.247403,0.039392,0.497692,0.018659,1452.021973,1.0,2,7
9053,0.464701,0.204894,0.068319,0.203080,0.098251,0.004644,0.023169,0.005235,448.785583,0.327284,...,0.033024,0.077684,0.193197,0.135930,0.552392,0.023126,1480.842529,1.0,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81703,0.017256,0.209403,0.070842,0.039287,0.403884,0.056886,0.163587,0.047513,5765.841309,0.003769,...,0.001737,0.002591,0.474666,0.046714,0.148483,0.035165,46012.910156,8.0,2,32
81704,0.039424,0.121447,0.025973,0.041864,0.234295,0.086562,0.322091,0.114735,22553.650391,0.005153,...,0.002619,0.003396,0.452980,0.049848,0.168879,0.045196,49227.437500,8.0,2,32
81705,0.032944,0.099493,0.029003,0.037000,0.186723,0.090095,0.391559,0.108849,27790.220703,0.004798,...,0.002605,0.003291,0.444350,0.050071,0.169784,0.040584,47833.875000,8.0,2,32
81706,0.006416,0.075112,0.029427,0.012989,0.182027,0.084335,0.456701,0.114316,24023.494141,0.004408,...,0.002769,0.003717,0.464063,0.049957,0.155699,0.030718,35271.335938,8.0,2,32


In [11]:
test_df['target'].max()

2

In [12]:
df = df[df.index.isin(test_df.index) == False]

In [13]:
df.shape

(83000, 210)

### train test split

In [14]:
df_interictal=df[df['target']==0]
df_interictal=df_interictal.sample(frac=0.2)
df_preictal=df[df['target']==2]
interictal_shape, preictal_shape = df_interictal.shape, df_preictal.shape
interictal_shape= df_interictal.shape

print(df_interictal.shape,  df_preictal.shape)


(14004, 210) (1468, 210)


In [15]:
X_interictal=np.array(df_interictal[df.columns[:-2]]).astype('float32')
y_interictal=np.array(df_interictal['target']).astype('float32')
X_preictal_train  =np.array(df_preictal[df.columns[:-2]]).astype('float32')
y_preictal_train  =np.array(df_preictal['target']).astype('float32')

In [16]:
print(X_interictal.shape)
print(y_interictal.shape)
print(X_preictal_train.shape)
print(y_preictal_train.shape)

(14004, 208)
(14004,)
(1468, 208)
(1468,)


### train test split

In [17]:
from sklearn.model_selection import train_test_split


In [18]:
X_interictal_train, X_interictal_test, y_interictal_train, y_interictal_test =train_test_split(X_interictal,y_interictal,test_size=0.1, random_state=42)
# X_preictal_train, X_preictal_test, y_preictal_train, y_preictal_test=train_test_split(X_preictal, y_preictal,test_size=0.1, random_state=42)

In [19]:
interictal_test_df=pd.DataFrame(data=X_interictal_test)
interictal_test_df['target']=y_interictal_test
interictal_test_df
# test_df=test_df.append(interictal_test_df, ignore_index=True)
test_df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,target,seizure_number
9049,0.487906,0.294157,0.062645,0.243455,0.061901,0.003636,0.013767,0.003427,733.779602,0.385181,...,0.066628,0.112517,0.115718,0.304856,0.541955,0.017563,1923.753174,1.0,2,7
9050,0.492928,0.287013,0.072026,0.238879,0.060910,0.002932,0.012587,0.004014,711.672302,0.415166,...,0.068790,0.102843,0.090541,0.174525,0.623085,0.014286,2015.595825,1.0,2,7
9051,0.406936,0.237590,0.078928,0.220020,0.104659,0.004607,0.020451,0.006975,398.291138,0.365368,...,0.043970,0.070681,0.184607,0.131209,0.573714,0.013855,1690.714233,1.0,2,7
9052,0.377607,0.232304,0.089268,0.232302,0.118813,0.004778,0.024522,0.006443,364.721436,0.288228,...,0.031371,0.090012,0.247403,0.039392,0.497692,0.018659,1452.021973,1.0,2,7
9053,0.464701,0.204894,0.068319,0.203080,0.098251,0.004644,0.023169,0.005235,448.785583,0.327284,...,0.033024,0.077684,0.193197,0.135930,0.552392,0.023126,1480.842529,1.0,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81703,0.017256,0.209403,0.070842,0.039287,0.403884,0.056886,0.163587,0.047513,5765.841309,0.003769,...,0.001737,0.002591,0.474666,0.046714,0.148483,0.035165,46012.910156,8.0,2,32
81704,0.039424,0.121447,0.025973,0.041864,0.234295,0.086562,0.322091,0.114735,22553.650391,0.005153,...,0.002619,0.003396,0.452980,0.049848,0.168879,0.045196,49227.437500,8.0,2,32
81705,0.032944,0.099493,0.029003,0.037000,0.186723,0.090095,0.391559,0.108849,27790.220703,0.004798,...,0.002605,0.003291,0.444350,0.050071,0.169784,0.040584,47833.875000,8.0,2,32
81706,0.006416,0.075112,0.029427,0.012989,0.182027,0.084335,0.456701,0.114316,24023.494141,0.004408,...,0.002769,0.003717,0.464063,0.049957,0.155699,0.030718,35271.335938,8.0,2,32


In [20]:
X_train = np.concatenate((X_interictal_train, X_preictal_train))
y_train = np.concatenate((y_interictal_train, y_preictal_train))
X_test = np.array(test_df[df.columns[:-2]]).astype('float32')
y_test  =np.array(test_df['target']).astype('float32')
X_test = np.concatenate((X_test, X_interictal_test))
y_test  =np.concatenate((y_test, y_interictal_test))

In [21]:
test_df

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,target,seizure_number
9049,0.487906,0.294157,0.062645,0.243455,0.061901,0.003636,0.013767,0.003427,733.779602,0.385181,...,0.066628,0.112517,0.115718,0.304856,0.541955,0.017563,1923.753174,1.0,2,7
9050,0.492928,0.287013,0.072026,0.238879,0.060910,0.002932,0.012587,0.004014,711.672302,0.415166,...,0.068790,0.102843,0.090541,0.174525,0.623085,0.014286,2015.595825,1.0,2,7
9051,0.406936,0.237590,0.078928,0.220020,0.104659,0.004607,0.020451,0.006975,398.291138,0.365368,...,0.043970,0.070681,0.184607,0.131209,0.573714,0.013855,1690.714233,1.0,2,7
9052,0.377607,0.232304,0.089268,0.232302,0.118813,0.004778,0.024522,0.006443,364.721436,0.288228,...,0.031371,0.090012,0.247403,0.039392,0.497692,0.018659,1452.021973,1.0,2,7
9053,0.464701,0.204894,0.068319,0.203080,0.098251,0.004644,0.023169,0.005235,448.785583,0.327284,...,0.033024,0.077684,0.193197,0.135930,0.552392,0.023126,1480.842529,1.0,2,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
81703,0.017256,0.209403,0.070842,0.039287,0.403884,0.056886,0.163587,0.047513,5765.841309,0.003769,...,0.001737,0.002591,0.474666,0.046714,0.148483,0.035165,46012.910156,8.0,2,32
81704,0.039424,0.121447,0.025973,0.041864,0.234295,0.086562,0.322091,0.114735,22553.650391,0.005153,...,0.002619,0.003396,0.452980,0.049848,0.168879,0.045196,49227.437500,8.0,2,32
81705,0.032944,0.099493,0.029003,0.037000,0.186723,0.090095,0.391559,0.108849,27790.220703,0.004798,...,0.002605,0.003291,0.444350,0.050071,0.169784,0.040584,47833.875000,8.0,2,32
81706,0.006416,0.075112,0.029427,0.012989,0.182027,0.084335,0.456701,0.114316,24023.494141,0.004408,...,0.002769,0.003717,0.464063,0.049957,0.155699,0.030718,35271.335938,8.0,2,32


In [22]:
X_train=np.array(X_train).astype('float32')
X_test=np.array( X_test).astype('float32')
y_train=np.array(y_train).astype('float32')
y_test=np.array( y_test).astype('float32')

X_train_shape =X_train.shape
X_test_shape = X_test.shape
y_train_shape =y_train.shape 
y_test_shape = y_test.shape
print(X_train_shape)
print(X_test_shape)
print(y_train_shape)
print(y_test_shape)

(14071, 208)
(1691, 208)
(14071,)
(1691,)


### Normalization

In [23]:
# from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# scaler.fit(X_train)
# scaler.transform(X_train)
# scaler.fit(X_test)
# scaler.transform(X_test)

### PCA

In [24]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=10)
# X_train2=pca.fit_transform(X_train)
# X_test2=pca.transform(X_test)

### training the model

In [25]:
from datetime import datetime

In [26]:
cost: 150.0
weight_dict: {0: 1, 2: 10}

In [27]:
before = datetime.now()
before_time =before.strftime("%H:%M:%S")
print(before_time)

clf = SVC(kernel='rbf', class_weight='balanced')
clf.fit(X_test, y_test)
trainAcc = clf.score(X_train, y_train)
testAcc = clf.score(X_test, y_test)
print("**SVM Results:**")
print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")

after = datetime.now()
after_time =after.strftime("%H:%M:%S")
print(after_time)

01:52:42


In [44]:
# import joblib

    # estimator = joblib.load("/my_models/%s.pkl"%dataset_name)
    # print "using trained model"

# print ("saving new model")
# joblib.dump(clf,"my_models/temporal5C10Wbalanced.pkl")

In [45]:
# loaded_clf = joblib.load("my_models/SVM_blanced_chb04.pkl")
# y_pred=loaded_clf.predict(X_test)
y_pred=clf.predict(X_test)

In [46]:
clf.get_params(deep=True)

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': 'balanced',
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'sigmoid',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [47]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_test, y_pred).ravel()
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))

TN:605, FP:796, FN:185, TP:105


In [48]:
specificity=(tn)/(tn+fp)
sensitivity=(tp)/(tp+fn)
print('specificity= {} , sensitivity= {}'.format(specificity, sensitivity))

specificity= 0.4318344039971449 , sensitivity= 0.3620689655172414


In [49]:
print('X size ={}, y size = {}'.format(X_shape, y_shape))
print('interictal size ={}, preictal size = {}'.format(interictal_shape, preictal_shape))
print('train size ={}, test size = {}'.format(X_train_shape, X_test_shape))

print("Training Accuracy: %d"%(trainAcc*100)+"%")
print("Testing Accuracy: %d"%(testAcc *100)+"%")
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))
print('specificity= {} , sensitivity= {}'.format(specificity, sensitivity))

X size =(83290, 208), y size = (83290,)
interictal size =(14004, 210), preictal size = (1468, 210)
train size =(14071, 208), test size = (1691, 208)
Training Accuracy: 45%
Testing Accuracy: 41%
TN:605, FP:796, FN:185, TP:105
specificity= 0.4318344039971449 , sensitivity= 0.3620689655172414


In [50]:
def f_importances(coef, names):
    imp = coef
    imp,names = zip(*sorted(zip(imp,names)))
    plt.barh(range(len(names)), imp, align='center')
    plt.yticks(range(len(names)), names)
    plt.show()
f_importances(clf.coef_, generate_column_names())

AttributeError: coef_ is only available when using a linear kernel

In [489]:
stop

NameError: name 'stop' is not defined

In [386]:
subject_id=10
base_path = "features_notwelch/"
# edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb01/*.npy".format(subject_id))))
# files=len(edf_file_names)
X_10=load('features_notwelch/data_chb{:02d}/features_{}_00.npy'.format(subject_id,subject_id))
y_10=load('features_notwelch/data_chb{:02d}/targets_{}_00.npy'.format(subject_id,subject_id))
for subject_id in range(10,11):
    edf_file_names = sorted(glob.glob(os.path.join(base_path, "data_chb{:02d}/*.npy".format(subject_id))))
    files=len(edf_file_names)
    start=0
    if subject_id==1:
        start=1
    for fileno in range(start, files//2):
        X_10=np.concatenate((X, load('features_notwelch/data_chb{:02d}/features_{}_{:02d}.npy'.format(subject_id, subject_id, fileno))))
        y_10=np.concatenate((y, load('features_notwelch/data_chb{:02d}/targets_{}_{:02d}.npy'.format(subject_id,subject_id, fileno))))
    print(files)
    print(X_10.shape)


50
(84000, 208)


In [404]:
test_10=clf.predict(X_10_test)
test_10.shape

(272,)

In [389]:
df10=pd.DataFrame(data=X_10, columns=generate_column_names())
# df['patient']=1
# df.at[14577:27187,'patient',]=2
# df.at[27187:40755, 'patient']=3
# df.at[40755:97783, 'patient']=4
# df.at[97783:, 'patient']=5

# df=df[np.append(df.columns[:50], ['rms', 'sum', 'patient'])]
df10['target']=y_10
print(df10.shape)
df10=df10.dropna()
print(df10.shape)
df10.reset_index(drop=True, inplace=True)
df10=df10[(df10['target']==0) | (df10['target']==2)]
print(df10['target'].max())
df10=df10[(df10['patient']==10) ]
df10
# X_10_test  =np.array(test_df[df.columns[-1]]).astype('float32')


(84000, 209)
(84000, 209)
2


,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-1,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient,target
83367,0.028653,0.158097,0.113682,0.082024,0.331338,0.053188,0.206389,0.049818,2155.678467,0.057258,...,0.178178,0.180902,0.110540,0.291462,0.041056,0.155702,0.045579,1393.498535,10.0,2
83368,0.033605,0.166255,0.136861,0.101153,0.301332,0.048791,0.184943,0.049261,2193.629883,0.076136,...,0.204738,0.221954,0.136177,0.332212,0.030593,0.091836,0.013840,1246.228271,10.0,2
83369,0.037503,0.161529,0.170949,0.121876,0.270961,0.046623,0.186121,0.042366,2193.087158,0.077694,...,0.128155,0.131862,0.077313,0.323281,0.038567,0.206293,0.075386,1935.947266,10.0,2
83370,0.039594,0.166173,0.153273,0.113643,0.280672,0.043745,0.192411,0.042643,2154.500977,0.062841,...,0.125706,0.149492,0.076814,0.302183,0.039134,0.210114,0.078373,1851.788330,10.0,2
83371,0.037242,0.184440,0.143269,0.112442,0.298192,0.045191,0.179890,0.039327,2077.309082,0.061768,...,0.204492,0.234095,0.155333,0.298474,0.030886,0.092033,0.013066,1146.315186,10.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83995,0.093291,0.111517,0.424273,0.332365,0.091842,0.010857,0.096643,0.030815,218.816040,0.096926,...,0.137944,0.246737,0.432356,0.106511,0.013906,0.046698,0.009977,247.024750,10.0,0
83996,0.176399,0.146374,0.351310,0.360416,0.059218,0.006547,0.066641,0.018250,283.695557,0.134067,...,0.118067,0.237217,0.534333,0.064199,0.006832,0.025440,0.005962,433.070465,10.0,0
83997,0.221352,0.118591,0.325421,0.449308,0.035077,0.004633,0.038773,0.009657,516.399658,0.117261,...,0.162081,0.228441,0.519111,0.052671,0.005097,0.018575,0.004661,581.740540,10.0,0
83998,0.195963,0.098899,0.322459,0.443545,0.042483,0.005407,0.047875,0.011504,404.965973,0.101507,...,0.187706,0.285315,0.428774,0.066840,0.008218,0.026593,0.007313,399.000153,10.0,0


In [390]:
df10['patient'].unique()

array([10.])

In [396]:
df10[df10.columns[:-1]]

,FP1-F7-0,FP1-F7-1,FP1-F7-2,FP1-F7-3,FP1-F7-4,FP1-F7-5,FP1-F7-6,FP1-F7-7,FP1-F7-8,F7-T7-0,...,T8-P8-1-0,T8-P8-1-1,T8-P8-1-2,T8-P8-1-3,T8-P8-1-4,T8-P8-1-5,T8-P8-1-6,T8-P8-1-7,T8-P8-1-8,patient
83367,0.028653,0.158097,0.113682,0.082024,0.331338,0.053188,0.206389,0.049818,2155.678467,0.057258,...,0.027383,0.178178,0.180902,0.110540,0.291462,0.041056,0.155702,0.045579,1393.498535,10.0
83368,0.033605,0.166255,0.136861,0.101153,0.301332,0.048791,0.184943,0.049261,2193.629883,0.076136,...,0.035381,0.204738,0.221954,0.136177,0.332212,0.030593,0.091836,0.013840,1246.228271,10.0
83369,0.037503,0.161529,0.170949,0.121876,0.270961,0.046623,0.186121,0.042366,2193.087158,0.077694,...,0.026962,0.128155,0.131862,0.077313,0.323281,0.038567,0.206293,0.075386,1935.947266,10.0
83370,0.039594,0.166173,0.153273,0.113643,0.280672,0.043745,0.192411,0.042643,2154.500977,0.062841,...,0.023689,0.125706,0.149492,0.076814,0.302183,0.039134,0.210114,0.078373,1851.788330,10.0
83371,0.037242,0.184440,0.143269,0.112442,0.298192,0.045191,0.179890,0.039327,2077.309082,0.061768,...,0.031906,0.204492,0.234095,0.155333,0.298474,0.030886,0.092033,0.013066,1146.315186,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
83995,0.093291,0.111517,0.424273,0.332365,0.091842,0.010857,0.096643,0.030815,218.816040,0.096926,...,0.109280,0.137944,0.246737,0.432356,0.106511,0.013906,0.046698,0.009977,247.024750,10.0
83996,0.176399,0.146374,0.351310,0.360416,0.059218,0.006547,0.066641,0.018250,283.695557,0.134067,...,0.123040,0.118067,0.237217,0.534333,0.064199,0.006832,0.025440,0.005962,433.070465,10.0
83997,0.221352,0.118591,0.325421,0.449308,0.035077,0.004633,0.038773,0.009657,516.399658,0.117261,...,0.163596,0.162081,0.228441,0.519111,0.052671,0.005097,0.018575,0.004661,581.740540,10.0
83998,0.195963,0.098899,0.322459,0.443545,0.042483,0.005407,0.047875,0.011504,404.965973,0.101507,...,0.174591,0.187706,0.285315,0.428774,0.066840,0.008218,0.026593,0.007313,399.000153,10.0


In [402]:
X_10_test  =np.array(df10[df10.columns[:-1]]).astype('float32')
y_10_test  =np.array(df10['target']).astype('float32')


In [403]:
tes_10_score = clf.score(X_10_test, y_10_test)
tes_10_score

0.7867647058823529

In [405]:
tn, fp, fn, tp = sklearn.metrics.confusion_matrix(y_10_test, test_10).ravel()
print("TN:{}, FP:{}, FN:{}, TP:{}".format(tn, fp, fn, tp))

TN:214, FP:0, FN:58, TP:0


### grid search

In [357]:
# from numpy import mean
# from sklearn.model_selection import GridSearchCV
# from sklearn.model_selection import RepeatedStratifiedKFold

In [358]:
# balance = [{0:10,2:1}, {0:1,2:1}, {0:1,2:10}, {0:1,2:50}, {0:1,2:100}]
# costs=[1.0,10.0,100.0]
# param_grid = dict(C=costs, class_weight=balance)

In [359]:
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=1, random_state=1)
# grid = GridSearchCV(estimator=clf, param_grid=param_grid, n_jobs=-1, cv=cv, scoring='f1_weighted')


In [360]:
# grid_result = grid.fit(X_train, y_train)

In [361]:
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
# means = grid_result.cv_results_['mean_test_score']
# stds = grid_result.cv_results_['std_test_score']
# params = grid_result.cv_results_['params']
# for mean, stdev, param in zip(means, stds, params):
#     print("%f (%f) with: %r" % (mean, stdev, param))